<a href="https://colab.research.google.com/github/jenish2/032_Jenish_Dholariya/blob/main/Lab_06_Logistic_Reg_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf

import numpy as np

Step 2: Loading and Preparing the MNIST Data Set

In [5]:
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Convert to float32.
x_train = np.array(x_train, np.float32)
x_test = np.array(x_test, np.float32)


# Flatten images to 1-D vector of 784 features (28*28).
num_features=784
x_train = x_train.reshape(-1,num_features)
x_test = x_test.reshape(-1,num_features)


# Normalize images value from [0, 255] to [0, 1].
x_train = x_train / 255
x_test = x_test /255


11501568/11490434 [==============================] - 0s 0us/step


In [6]:
print(f'training data {x_train.shape}')
print(f'test data {x_test.shape}')
print(f'training label {y_train.shape}')
print(f'testing label {y_train.shape}')

training data (60000, 784)
test data (10000, 784)
training label (60000,)
testing label (60000,)


#Step 3: Setting Up Hyperparameters and Data Set Parameters

Initialize the model parameters. 

num_classes denotes the number of outputs, which is 10, as we have digits from 0 to 9 in the data set. 

num_features defines the number of input parameters, and we store 784 since each image contains 784 pixels.

In [7]:
# MNIST dataset parameters.

num_classes = 10 # 0 to 9 digits

num_features = 784 # 28*28

# Training parameters.

learning_rate = 0.01

training_steps = 1000

batch_size = 256

display_step = 50

#Step 4: Shuffling and Batching the Data

We need to shuffle and batch the data before we start the actual training to avoid the model from getting biased by the data. This will allow our data to be more random and helps our model to gain higher accuracies with the test data.

With the help of tf.data.Dataset.from_tensor_slices, we can get the slices of an array in the form of objects. 

The function shuffle(5000) randomizes the order of the data set’s examples. 

Here, 5000 denotes the variable shuffle_buffer, which tells the model to pick a sample randomly from 1 to 5000 samples. 

After that, there are only 4999 samples left in the buffer, so the sample 5001 gets added to the buffer. 

In [8]:
# Use tf.data API to shuffle and batch data.
num_batches = int(x_train.shape[0]/batch_size)

train_data=tf.data.Dataset.from_tensor_slices((x_train,y_train))

train_data=train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

#Step 5: Initializing Weights and Biases

We now initialize the weights vector and bias vector with ones and zeros.

In [9]:
# Weight of shape [784, 10], the 28*28 image features, and a total number of classes.

W = tf.Variable(np.random.randn(784,10).astype(np.float32))
# X = tf.placeholder(tf.float32,[None,784])
# Y = tf.placeholder(tf.float32, [None,10])

# Bias of shape [10], the total number of classes.
B = tf.Variable(np.random.randn(10).astype(np.float32))

#Step 6: Defining Logistic Regression and Cost Function

We define the logistic_regression function below, which converts the inputs into a probability distribution proportional to the exponents of the inputs using the softmax function. The softmax function, which is implemented using the function tf.nn.softmax, also makes sure that the sum of all the inputs equals one.

In [10]:
# Logistic regression (Wx + b).

def logistic_regression(x):

    # Apply softmax to normalize the logits to a probability distribution.
    return tf.nn.softmax(tf.add(tf.matmul(x,W),B))
    

# Cross-Entropy loss function.

def cross_entropy(y_pred, y_true):

    # Encode label to a one hot vector.
    y_true = tf.one_hot(y_true, depth=num_classes)
    

    # Clip prediction values to avoid log(0) error.

    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)    

    # Compute cross-entropy.
    ce_loss = tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))
    return ce_loss

#Step 7: Defining Optimizers and Accuracy Metrics
When we compute the output, it gives us the probability of the given data to fit a particular class of output. 

We consider the correct prediction as to the class having the highest probability. 

We compute this using the function tf.argmax. 

We also define the stochastic gradient descent as the optimizer from several optimizers present in TensorFlow. We do this using the function tf.optimizers.SGD. 

This function takes in the learning rate as its input, which defines how fast the model should reach its minimum loss or gain the highest accuracy.

In [11]:
# Accuracy metric.

def accuracy(y_pred, y_true):

  # Predicted class is the index of the highest score in prediction vector (i.e. argmax).

  correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))

  return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


#Step 8: Optimization Process and Updating Weights and Biases
Now we define the run_optimization() method where we update the weights of our model. We calculate the predictions using the logistic_regression(x) method by taking the inputs and find out the loss generated by comparing the predicted value and the original value present in the data set. Next, we compute the gradients using and update the weights of the model with our stochastic gradient descent optimizer.

In [12]:
# Optimization process. 

optimizer = tf.optimizers.SGD(learning_rate)

def run_optimization(x, y):

# Wrap computation inside a GradientTape for automatic differentiation.

    with tf.GradientTape() as g:

        pred = logistic_regression(x)

        loss = cross_entropy(pred, y)

    # Compute gradients.

    gradients = g.gradient(loss, [W, B])


    # Stochastic gradient descent optimizer.

    
    
    # Update W and b following gradients.

    optimizer.apply_gradients(zip(gradients, [W, B]))

#Step 9: The Training Loop

In [13]:
# Run training for the given number of steps.

for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):

    # Run the optimization to update W and b values.

    run_optimization(batch_x, batch_y)

    

    if step % display_step == 0:

        #Obtain Predictions
        pred = logistic_regression(batch_x)
        #Ccompute loss
        loss = cross_entropy(pred, batch_y)
        #Compute Accuracy
        acc = accuracy(pred, batch_y)
        #print accuracy
        print(f"step: {step}, loss: {loss}, accuracy: {acc}")

step: 50, loss: 178.59625244140625, accuracy: 0.87109375
step: 100, loss: 191.7235107421875, accuracy: 0.83984375
step: 150, loss: 88.9710693359375, accuracy: 0.90234375
step: 200, loss: 172.49630737304688, accuracy: 0.8203125
step: 250, loss: 109.13772583007812, accuracy: 0.92578125
step: 300, loss: 152.96878051757812, accuracy: 0.8671875
step: 350, loss: 123.51658630371094, accuracy: 0.88671875
step: 400, loss: 150.00352478027344, accuracy: 0.87109375
step: 450, loss: 106.1557846069336, accuracy: 0.921875
step: 500, loss: 122.88339233398438, accuracy: 0.90234375
step: 550, loss: 117.80401611328125, accuracy: 0.90625
step: 600, loss: 68.46585083007812, accuracy: 0.94140625
step: 650, loss: 75.74462890625, accuracy: 0.921875
step: 700, loss: 58.93746566772461, accuracy: 0.93359375
step: 750, loss: 136.08070373535156, accuracy: 0.890625
step: 800, loss: 41.57963180541992, accuracy: 0.96484375
step: 850, loss: 68.01307678222656, accuracy: 0.92578125
step: 900, loss: 56.48239517211914, ac

#Step 10: Testing Model Accuracy Using the Test Data

Finally, we check the model accuracy by sending the test data set into our model and compute the accuracy using the accuracy function that we defined earlier.

In [14]:
# Test model on validation set.
pred = logistic_regression(x_test)
accu = accuracy(pred, y_test)
print(f"Test Accuracy: {accu}")

Test Accuracy: 0.9023000001907349
